# Introduction
This code is an MCMC fit for sine curves using the emcee package (https://emcee.readthedocs.io/en/stable/). The likelihood function, priors, and random additions are set up for you already. 

### You will be changing:

1.) The initial values of the algorithm - think about how you can find these analytically based on the data that you have

2.) The number of steps the algorithm takes - think about how the number of steps you take affects the quality of your fit

3.) The generated error - think about how error changes how percice your fit can be

### A few thing to note on the items that are being plotted or printed:

1.) Trace: this shows how much your values are moving around during your fit, so how much variation you're getting. The more jagged the plot is, the more variation you get. Note the very jagged part right at the beginning, that's your burn-in. The burn-in is your MCMC figuring out what value it wants to settle on before getting into a holding pattern on that value. It is usually discarded in the next plots.

2.) Corner Plots: these show how constrained the fit is. Good corner plots should look something like this: (https://www.researchgate.net/figure/Corner-plot-resulting-from-our-MCMC-analysis-of-the-TESS-photometry-of-14-Peg-Red_fig1_347534572), where there's a good spread of values that cluster around a central point, which is our correct value. 

3.) The MCMC's Answer: these values are the answers from MCMC with errors. Our errors are calculated by getting the difference between our 50th percentile answer and our 16th percentile answer, and our 84th percentile answer and our 50th percentile answer for our lower error bar and upper error bar, respectively.

4.) Our Final Fitted Plot: note that I declare the model within the plt.plot() call itself. You might want to change this later depending on how you use this algorithm.

### Final Notes:
Values you can and should change are marked with a comment with two exclamation points. Running a 5000 step run takes about three seconds for me, it should take around that amount of time for you. Once you've gotten used to playing around with a generated fit and you've gotten used to sine fitting TOI-4468 without using MCMC, switch over to using the RV data for TOI-4468. It'll provide some interesting challenges, especially given that you will have to phase fold before you input it into the MCMC run.

A few important notes about MCMC. Depending on how your traces look, you might need to throw away more steps in your burn-in. Look for where that the line flattens out, that's how many steps you'll want to throw away. The farther away from the real values your fit is, the larger your burn-in. In MCMC, your fit will be improved the more steps you add, to a point. At a certain point, if you add more steps, you're just wasting time. Since these values have errors on them, we can only be so accurate in our knowledge of the "true value." This is just an approximation, a very good one, but still just an approximation. Keep that in mind before you ask it to run a billion steps. Finally, when you use this code for TOI-4468, I would suggest making a copy of this notebook and editing that so you still have whatever edits you've made here along with the new work you're going to do. I will always have this original copy of the notebook, so if you need it, it's always here.

Hope this is helpful, good luck, and happy fitting!

In [1]:
# Import statements - I use a lot, don't worry about these
import numpy as np

import matplotlib.pyplot as plt

from scipy.optimize import minimize

import emcee

import corner

from IPython.display import display, Math

import matplotlib as mpl
# mpl.rc('font', family='serif', serif='Times New Roman')
# mpl.rcParams['figure.dpi'] = 600
import seaborn as sns
from IPython.core.display import display, HTML
# display(HTML("<style>.container { width:96% !important; }</style>"))
%matplotlib inline
import glob 
import pandas as pd
pd.options.display.max_columns = 100
pd.options.display.max_rows = 100

# plt.rcParams.update({'figure.figsize': (12,6)})
#from astroquery.simbad import Simbad
#from astroquery.vizier import Vizier
#from astroquery.mast import Observations
#import astropy.units as u
#import lightkurve as lk
import warnings
warnings.filterwarnings('ignore')

import scipy.stats as stats

#from copy import deepcopy
#from astropy import table

#from astropy.io import fits

/var/folders/v3/t2tf85qd2rd8bv26z7c0rr7w0000gn/T/ipykernel_60565/860263722.py:18: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [2]:
# Define the sine function we'll use as our model
def sin_function(t, amp, freq, pha, vert):
    return amp * np.sin((freq * t) + pha) + vert
    
def fit_function(t, amp, pha, vert):
    return amp * np.sin((2*np.pi * t) + pha) + vert

In [3]:
# Define Priors - The boundaries on what is allowed for our model
## Returning 0 shows that the values are valid for the model
## Returning -infinity shows that the values are not valid for the model
def log_prior(theta):
    ## Get our parameters for the model
    amplitude, phase, vertical_shift = theta
    
    ## Check if the amplitude and frequency are greater than 0
    if amplitude > 0 and 2*np.pi > phase > 0: # !! Change these if you want a more constrained fit
        return 0
    
    return -np.inf

In [4]:
# Define Likelihood Function
def log_likelihood(parameters, x, y, data_err):
    ## Get our parameters for the model
    amplitude, phase, vertical_shift = parameters
    
    ## Check our parameters against our priors
    ## If lp is negative infinity, the values are invalid, a model will not be calculated, 
    ##  and these values will be labeled as very unlikely
    lp = log_prior(parameters)
    if not np.isfinite(lp):
        return -np.inf
    
    ## Generate a new version of our model using our current x values
    model = fit_function(x, amplitude, phase, vertical_shift)
    
    
    ## Compare using chi squared
    chisquared = np.nansum(((y-model)**2/(data_err**2)))
    ## Return our chi squared value for later evaluation done by emcee
    return (-0.5 * chisquared)

In [5]:
# Set up initial values with random additions
ndim = 3
nwalkers = 16 # !! Change this if you want more walkers looking around parameter space. This will make your fit take longer.

# Creates an array of random additions the same size as our number of walkers and number of values being fit
random_additions = np.random.rand(nwalkers, ndim)

# Sets random additions to be a small value for a small variation
## !! Feel free to change the 0.01 values after random_additions[i][j]. I've marked them with a comment with '!!'
##  These will change how far your values change on each step of the MCMC run
for i in range(len(random_additions)):
    for j in range(len(random_additions[0])):
        if j == 0:
            random_additions[i][j] = random_additions[i][j] * 0.01 # !!
        elif j == 1:
            random_additions[i][j] = random_additions[i][j] * 0.01 # !!
        elif j == 2:
            random_additions[i][j] = random_additions[i][j] * 0.01 # !!
        #elif j == 3:
        #    random_additions[i][j] = random_additions[i][j] * 0.01 # !!

# Initial Values - Insert your own way to find the initial amplitude, frequency, phase, and vertical shift!
amp0 = 32.51286944051743 # !!
phase0 = 2.025620444551217 # !!
vert0 = -15.502804859016733 # !!

# !! Declare an array with our initial conditions
p0 = np.add(np.asarray([amp0, phase0, vert0]), random_additions)

In [6]:
# Generate data with noise and errors
#t = np.linspace(0, 4 * np.pi, 8)

# !! Change these values for a different sine curve!
#amp_true, freq_true, phase_true, vert_true = 10, 1.5, 2.5, 0
#sinecurve = sin_function(t, amp_true, freq_true, phase_true, vert_true) + np.random.normal(0, 0.7, t.size)

# !! Change the magnitude of errors by changing the second value in this call here
#gen_err = np.random.normal(0.1, 0.3, t.size)

In [7]:
# Reading of Data
NEID = pd.read_csv("updated_rvs.csv")

# Processing Radial Velocity
RV = NEID.iloc[:, 1]
print(RV)

# Processing Time in BJD
BJD = NEID.iloc[:, 0] #BJD(days)
print(BJD)

# Processing Error
dvrms = NEID.iloc[:, 2]
print(dvrms)


0     54.432074
1    -21.713454
2    -45.167308
3     -5.299056
4    -39.979129
5    -10.046810
6    -47.531130
7    -54.320239
8    133.349369
9    -33.789008
Name: rv, dtype: float64
0    2.460195e+06
1    2.460237e+06
2    2.460240e+06
3    2.460328e+06
4    2.460329e+06
5    2.460342e+06
6    2.460412e+06
7    2.460448e+06
8    2.460455e+06
9    2.460473e+06
Name: bjd, dtype: float64
0     7.408624
1     9.791496
2     8.761311
3     6.358875
4    10.720334
5    10.144447
6     8.128676
7    10.142950
8     7.554475
9     6.227334
Name: e_rv, dtype: float64


In [8]:
# Phase Folding
period_C = 7.014001 #Days
phase_C = (BJD % period_C) / period_C


In [ ]:
# Run Emcee!
## Declare the sampler
sampler = emcee.EnsembleSampler(
    ## Tells the sampler how many walkers are wanted (nwalkers), how many values are being fitted (ndim),
    ##  our likelihood function, and the constants used in the fit/arguments (args=(t, sinecurve, gen_err)) 
    nwalkers, ndim, log_likelihood, args=(phase_C, RV, dvrms)
)
# Runs the sampler given our initial conditions (p0)
## Set to 5000 steps for a quick fit. Additionally shows a progress bar so you can see how long it will take to run
## !! Change number of steps for a more percice fit (to a point!)
sampler.run_mcmc(p0, 50000, progress=True);

 99%|███████████████████████████████████▊| 49741/50000 [02:12<00:00, 372.17it/s]

In [ ]:
# Plot Trace
fig, axes = plt.subplots(ndim, figsize=(10, 9), sharex=True)
samples = sampler.get_chain()
labels = ['Amplitude', 'Phase', 'Vertical Shift']
for i in range(ndim):
    ax = axes[i]
    ax.plot(samples[:, :, i], "k", alpha=0.3)
    ax.set_xlim(0, len(samples))
    ax.set_ylabel(labels[i])
    ax.yaxis.set_label_coords(-0.1, 0.5)

axes[-1].set_xlabel("step number");

In [ ]:
# Discard first 100 walkers
flat_samples = sampler.get_chain(discard=100, thin=15, flat=True) # !! Change discard=XXX depending on how traces look
print(flat_samples.shape)

In [ ]:
# Plot corner plots
fig = corner.corner(
    flat_samples, labels=labels
);

In [ ]:
# Print calculated values
answers = []
for i in range(ndim):
    mcmc = np.percentile(flat_samples[:, i], [16, 50, 84])
    q = np.diff(mcmc)
    txt = "\mathrm{{{3}}} = {0:.7f}_{{-{1:.7f}}}^{{{2:.7f}}}"
    txt = txt.format(mcmc[1], q[0], q[1], labels[i])
    answers.append(mcmc[1])
    display(Math(txt))

In [ ]:
ampmax = answers[0] + err[0]
ampmin = answers[0] - err[1]

phasemax = answers[1] + err[2]
phasemin = answers[1] - err[3]


vertmax = answers[2] + err[4]
vertmin = answers[2] - err[5]

In [ ]:
# Plot data and fitted curve
plt.figure(figsize=(12, 6))
t_fit = np.linspace(0, 1, 100)
phase_fit = fit_function(t_fit, answers[0], answers[1], answers[2])
phase_fit_max = fit_function(t_fit, ampmax, phasemax,vertmax)
phase_fit_min = fit_function(t_fit, ampmin, phasemin,vertmin)
plt.plot(t_fit, phase_fit, label='Fitted Sine Curve, Highest Possibility', color='tab:red', linewidth=3)
plt.plot(t_fit, phase_fit_max, label='Fitted Sine Curve, Max Value', color='tab:blue', linewidth=3)
plt.plot(t_fit, phase_fit_min, label='Fitted Sine Curve, Min Value', color='tab:purple', linewidth=3)
plt.errorbar(phase_C, RV, yerr=abs(dvrms), label='RV Data and Errors', fmt='o', capsize=5)
plt.xlabel('Phase of Orbit')
plt.ylabel('Radial Velocity')
plt.title('Data Fitting to a Sine Curve Using MCMC')
plt.legend()
plt.grid()
plt.show()

In [ ]:
#Mass Computation
import numpy as np

# Given parameters
K = 32.5195996-3.3163424 # Radial velocity amplitude in m/s
P = 606009.686  # Orbital period in seconds
M_s = 1.63489e30  # Mass of the host star in kg
e = 0  # Eccentricity

# Gravitational constant in m^3 kg^-1 s^-2
G = 6.67430e-11  

# Calculate M_p * sin(i)
Mp_sin_i = K * ((P / (2 * np.pi * G)) ** (1/3)) * (M_s ** (2/3)) * (1 / np.sqrt(1 - e**2))

# Output the result
Mp_sin_i

Mp_sin_i = Mp_sin_i / 1.898e27
print(Mp_sin_i)